In [22]:
import gpas_covid_synthetic_reads as gcsr
import gumpy, pandas, numpy

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
primer_scheme_file = 'gpas_covid_synthetic_reads/data/midnight-1200.qcovid.tsv'
primers=pandas.read_csv(primer_scheme_file,\
                            sep='\t',
                            names=['pool','name','seq','bool1','bool2','start'])
def assign_end(row):
            return(row['start']+len(row.seq)-1)

primers['end']=primers.apply(assign_end,axis=1)

def find_handedness(row):
    return(row['name'].split('_')[-1])

primers['hand']=primers.apply(find_handedness,axis=1)

left=primers.loc[primers.hand=='LEFT']
right=primers.loc[primers.hand=='RIGHT']

amplicons=left.merge(right,left_on='pool',right_on='pool',suffixes=['_left','_right'])
amplicons=amplicons[['pool','start_left','end_right']]
amplicons.rename(columns={'pool':'name','start_left':'start','end_right':'end'},inplace=True)

In [3]:
primers[:3]

,pool,name,seq,bool1,bool2,start,end,hand
0,SARSCoV_1200_1,SARSCoV_1200_1_LEFT,ACCAACCAACTTTCGATCTCTTGT,True,True,30,53,LEFT
1,SARSCoV_1200_1,SARSCoV_1200_1_RIGHT,GGTTGCATTCATTTGGTGACGC,False,False,1205,1226,RIGHT
2,SARSCoV_1200_3,SARSCoV_1200_3_LEFT,GGCTTGAAGAGAAGTTTAAGGAAGGT,True,True,2153,2178,LEFT


In [10]:
covid_reference=gumpy.Genome('gpas_covid_synthetic_reads/data/MN908947.3.gbk')
covid_reference.nucleotide_index

array([    1,     2,     3, ..., 29901, 29902, 29903])

In [42]:
row

,name,start,end,length
8,SARSCoV_1200_17,16624,17775,1151


In [60]:
mask

array([False, False, False, ..., False, False, False])

In [64]:
sum(mask)

0

In [23]:
a = numpy.array([False, False, True, True, True, False, False])
b = numpy.array([False, True,  True, False, False, False, False])

In [26]:
a * b

array([False, False,  True, False, False, False, False])

In [75]:
chosen_amplicon = 'SARSCoV_1200_17'
row = amplicons[amplicons.name == chosen_amplicon]

mask = (covid_reference.nucleotide_index >= int(row['start'])) & (covid_reference.nucleotide_index < int(row['end']))
print(numpy.sum(mask))

for idx, row in amplicons.iterrows():

    if row['name'] == chosen_amplicon:
        continue
    
    current_amplicon_mask = (covid_reference.nucleotide_index >= int(row['start'])) & (covid_reference.nucleotide_index < int(row['end']))
    
    overlap_region = current_amplicon_mask & mask
    
    overlap_region = numpy.logical_not(overlap_region)
    
    if numpy.sum(overlap_region) != 29903:
        print(row['name'], numpy.sum(overlap_region))
    
    mask = overlap_region & mask

print(numpy.sum(mask))

covid_reference.nucleotide_sequence[mask] = 'n'

1151
SARSCoV_1200_16 29786
SARSCoV_1200_18 29750
881


In [76]:
covid_reference.nucleotide_sequence[mask]

array(['n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n',
       'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n',
       'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n',
       'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n',
       'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n',
       'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n',
       'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n',
       'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n',
       'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n',
       'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n',
       'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n',
       'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n',
       'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n',
       'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n

In [63]:
sum(overlap_region)

29141

In [35]:
covid_reference.nucleotide_index[mask]

array([16624, 16625, 16626, ..., 17772, 17773, 17774])

In [4]:
amplicons['length'] = amplicons['end']-amplicons['start']

In [5]:
amplicons

,name,start,end,length
0,SARSCoV_1200_1,30,1226,1196
1,SARSCoV_1200_3,2153,3278,1125
2,SARSCoV_1200_5,4167,5380,1213
3,SARSCoV_1200_7,6283,7422,1139
4,SARSCoV_1200_9,8253,9421,1168
5,SARSCoV_1200_11,10343,11490,1147
6,SARSCoV_1200_13,12450,13642,1192
7,SARSCoV_1200_15,14540,15756,1216
8,SARSCoV_1200_17,16624,17775,1151
9,SARSCoV_1200_19,18596,19700,1104


In [27]:
lineages_reference=gcsr.load_lineages_dataframe()
lineages_reference[:3]

,pango_lineage,who_label
0,cB.1.1.7,Alpha
1,cB.1.351,Beta
2,cP.1,Gamma


In [26]:
pango_definitions=gcsr.load_pango_definitions('../constellations/', lineages_reference)
pango_definitions['cB.1.1.7']

{'label': 'Alpha (B.1.1.7-like)',
 'description': 'B.1.1.7 lineage defining mutations',
 'sources': ['https://virological.org/t/563'],
 'type': 'variant',
 'variant': {'Pango_lineages': ['B.1.1.7'],
  'mrca_lineage': 'B.1.1.7',
  'PHE_label': 'VOC-20DEC-01',
  'WHO_label': 'Alpha',
  'representative_genome': ''},
 'tags': ['B.1.1.7', 'VOC 202012/01'],
 'sites': ['nuc:C913T',
  '1ab:T1001I',
  '1ab:A1708D',
  'nuc:C5986T',
  '1ab:I2230T',
  '1ab:SGF3675-',
  'nuc:C14676T',
  'nuc:C15279T',
  'nuc:T16176C',
  's:HV69-',
  's:Y144-',
  's:N501Y',
  's:A570D',
  's:P681H',
  's:T716I',
  's:S982A',
  's:D1118H',
  'nuc:C26801T',
  '8:Q27*',
  '8:R52I',
  '8:Y73C',
  'N:D3L',
  'N:S235F'],
 'rules': {'min_alt': 15, 'max_ref': 3}}

array([    1,     2,     3, ..., 29901, 29902, 29903])

In [2]:
variant_definitions=gcsr.load_variant_definitions('../variant_definitions/')

In [4]:
omicron=gcsr.VariantGenome(covid_reference,variant_definitions['Omicron'])

In [5]:
omicron.variant

MN908947
MN908947.3
Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome
29903 bases
attaaa...aaaaaa
metadata for all genes/loci have been included

In [8]:
diff = covid_reference - omicron.variant

In [11]:
import numpy

In [12]:
diff.nucleotide_index

array([ 2832,  8393, 10449, 13195, 15240, 18163, 22578, 22673, 22674,
       22679, 22813, 22882, 22898, 22992, 23013, 23040, 23048, 23055,
       23202, 23525, 23599, 25000, 26530, 26577, 26709, 27259, 27807,
        6513, 11287, 21765, 21987])

In [13]:
covid_reference.nucleotide_sequence[numpy.isin(covid_reference.nucleotide_index,diff.nucleotide_index)]

array(['a', 'g', 'g', 'c', 'g', 't', 'c', 'a', 't', 'g', 'g', 't', 'c',
       't', 'g', 't', 'g', 'g', 'a', 'a', 'g', 'a', 'c', 'c', 't', 'c',
       'a', 'c', 'g', 'a', 'c'], dtype='<U1')

In [14]:
covid_reference.genes

{'orf1ab': {'reverse_complement': False,
  'type': 'GENE',
  'codes_protein': True,
  'start': 266,
  'end': 21556,
  'ribosomal_shifts': [13202]},
 'S': {'reverse_complement': False,
  'type': 'GENE',
  'codes_protein': True,
  'start': 21563,
  'end': 25385,
  'ribosomal_shifts': []},
 'ORF3a': {'reverse_complement': False,
  'type': 'GENE',
  'codes_protein': True,
  'start': 25393,
  'end': 26221,
  'ribosomal_shifts': []},
 'E': {'reverse_complement': False,
  'type': 'GENE',
  'codes_protein': True,
  'start': 26245,
  'end': 26473,
  'ribosomal_shifts': []},
 'M': {'reverse_complement': False,
  'type': 'GENE',
  'codes_protein': True,
  'start': 26523,
  'end': 27192,
  'ribosomal_shifts': []},
 'ORF6': {'reverse_complement': False,
  'type': 'GENE',
  'codes_protein': True,
  'start': 27202,
  'end': 27388,
  'ribosomal_shifts': []},
 'ORF7a': {'reverse_complement': False,
  'type': 'GENE',
  'codes_protein': True,
  'start': 27394,
  'end': 27760,
  'ribosomal_shifts': []},
 

In [15]:

for gene in covid_reference.genes:
    v=omicron.variant.build_gene(gene)
    r=covid_reference.build_gene(gene)
    d=r-v
    print(gene, d.mutations)

orf1ab ['K856R' 'A2710T' 'P3395H' 'V4310V' 'N4992N' 'I5967V' '6248_del_gtt'
 '11022_del_gtctggttt']
S ['G339D' 'S371L' 'S373P' 'K417N' 'N440K' 'G446S' 'S477N' 'E484A' 'Q493R'
 'G496S' 'Q498R' 'T547K' 'H655Y' 'N679K' 'D1146D' '203_del_tacatg'
 '425_del_gtgtttatt']
ORF3a []
E []
M ['D3G' 'Q19E' 'A63T']
ORF6 ['R20R']
ORF7a []
ORF8 ['c-87t']
N []
ORF10 []


In [18]:
d.mutations

array([], dtype=float64)

In [16]:
import json

In [20]:
with open('constellations/constellations/definitions/cB.1.1.529.json') as INPUT:
    a=json.load(INPUT)

In [22]:
a.keys()

dict_keys(['label', 'description', 'sources', 'type', 'variant', 'tags', 'sites', 'note', 'rules'])

In [51]:
ref=gumpy.Genome('gpas_covid_synthetic_reads/data/MN908947.3.gbk')
ref2=gumpy.Genome('gpas_covid_synthetic_reads/data/NC_045512.2.gbk')

In [56]:
seq=''.join(i for i in ref.nucleotide_sequence)
seq2=''.join(i for i in ref2.nucleotide_sequence)

In [57]:
seq==seq2

True

In [59]:
ref2.genes.keys()

dict_keys(['ORF1ab', 'ORF1ab_2', 'S', 'ORF3a', 'E', 'M', 'ORF6', 'ORF7a', 'ORF7b', 'ORF8', 'N', 'ORF10'])

In [39]:
ref.genes.keys()

dict_keys(['orf1ab', 'S', 'ORF3a', 'E', 'M', 'ORF6', 'ORF7a', 'ORF8', 'N', 'ORF10'])

In [ ]:
genes_lookup={'orf1ab': 'orf1ab',
              'spike': 'S',
              'e': 'E',
              'm': 'M',
              }

In [38]:
a

{'label': 'Omicron (B.1.1.529-like)',
 'description': 'B.1.1.529 lineage defining mutations',
 'sources': [],
 'type': 'variant',
 'variant': {'Pango_lineages': ['B.1.1.529'],
  'WHO_label': 'Omicron',
  'mrca_lineage': 'B.1.1.529',
  'representative_genome': '',
  'lineage_name': 'B.1.1.529',
  'incompatible_lineage_calls': ['BA.1', 'BA.2', 'BA.3']},
 'tags': ['B.1.1.529'],
 'sites': ['nuc:C3037T',
  'orf1ab:T3255I',
  'orf1ab:P3395H',
  'orf1ab:SGF3675-',
  'nuc:T13195C',
  'orf1ab:P4715L',
  'orf1ab:I5967V',
  'spike:G142D',
  'spike:G339D',
  'spike:S373P',
  'spike:S375F',
  'spike:K417N',
  'spike:N440K',
  'spike:S477N',
  'spike:T478K',
  'spike:E484A',
  'spike:Q493R',
  'spike:Q498R',
  'spike:N501Y',
  'spike:Y505H',
  'spike:D614G',
  'spike:H655Y',
  'spike:N679K',
  'spike:P681H',
  'spike:N764K',
  'spike:D796Y',
  'spike:Q954H',
  'spike:N969K',
  'nuc:C25000T',
  'nuc:C25584T',
  'e:T9I',
  'm:Q19E',
  'm:A63T',
  'nuc:A27259C',
  'nuc:C27807T',
  'nuc:A28271T',
  'n:P

In [69]:
a

{'label': 'Omicron (B.1.1.529-like)',
 'description': 'B.1.1.529 lineage defining mutations',
 'sources': [],
 'type': 'variant',
 'variant': {'Pango_lineages': ['B.1.1.529'],
  'WHO_label': 'Omicron',
  'mrca_lineage': 'B.1.1.529',
  'representative_genome': '',
  'lineage_name': 'B.1.1.529',
  'incompatible_lineage_calls': ['BA.1', 'BA.2', 'BA.3']},
 'tags': ['B.1.1.529'],
 'sites': ['nuc:C3037T',
  'orf1ab:T3255I',
  'orf1ab:P3395H',
  'orf1ab:SGF3675-',
  'nuc:T13195C',
  'orf1ab:P4715L',
  'orf1ab:I5967V',
  'spike:G142D',
  'spike:G339D',
  'spike:S373P',
  'spike:S375F',
  'spike:K417N',
  'spike:N440K',
  'spike:S477N',
  'spike:T478K',
  'spike:E484A',
  'spike:Q493R',
  'spike:Q498R',
  'spike:N501Y',
  'spike:Y505H',
  'spike:D614G',
  'spike:H655Y',
  'spike:N679K',
  'spike:P681H',
  'spike:N764K',
  'spike:D796Y',
  'spike:Q954H',
  'spike:N969K',
  'nuc:C25000T',
  'nuc:C25584T',
  'e:T9I',
  'm:Q19E',
  'm:A63T',
  'nuc:A27259C',
  'nuc:C27807T',
  'nuc:A28271T',
  'n:P

In [68]:
for i in a['sites']:
    cols = i.split(':')
    if cols[0] == 'nuc':
        print('mutate genome, easy')
    elif cols[0] == 'del':
        print('delete bases from genome, fairly easy')
    elif cols[0].upper() in constellation_genome['genes'].keys():
        print('mutation in gene, harder')
    else:
        print(i)

#     assert len(cols) == 2, i
# mutation='spike:H655Y'


mutate genome, easy
orf1ab:T3255I
orf1ab:P3395H
orf1ab:SGF3675-
mutate genome, easy
orf1ab:P4715L
orf1ab:I5967V
spike:G142D
spike:G339D
spike:S373P
spike:S375F
spike:K417N
spike:N440K
spike:S477N
spike:T478K
spike:E484A
spike:Q493R
spike:Q498R
spike:N501Y
spike:Y505H
spike:D614G
spike:H655Y
spike:N679K
spike:P681H
spike:N764K
spike:D796Y
spike:Q954H
spike:N969K
mutate genome, easy
mutate genome, easy
mutation in gene, harder
mutation in gene, harder
mutation in gene, harder
mutate genome, easy
mutate genome, easy
mutate genome, easy
mutation in gene, harder
delete bases from genome, fairly easy
mutation in gene, harder


In [49]:
with open('constellations/constellations/data/SARS-CoV-2.json') as f:
    constellation_genome=json.load(f)

In [66]:
constellation_genome['genes'].keys()

dict_keys(['ORF1a', 'ORF1b', 'S', 'ORF3a', 'E', 'M', 'ORF6', 'ORF7a', 'ORF8', 'N', 'ORF10'])

In [61]:
constellation_genome['proteins']

{'NSP1': {'name': 'NSP1',
  'gene': '1ab',
  'description': 'leader protein',
  'coordinates': {'from': 1, 'to': 180}},
 'NSP2': {'name': 'NSP2',
  'gene': '1ab',
  'coordinates': {'from': 181, 'to': 818}},
 'NSP3': {'name': 'NSP3',
  'gene': '1ab',
  'coordinates': {'from': 819, 'to': 2763}},
 'NSP4': {'name': 'NSP4',
  'gene': '1ab',
  'coordinates': {'from': 2764, 'to': 3263}},
 'NSP5': {'name': 'NSP5',
  'gene': '1ab',
  'description': '3C-like proteinase',
  'coordinates': {'from': 3264, 'to': 3569}},
 'NSP6': {'name': 'NSP6',
  'gene': '1ab',
  'coordinates': {'from': 3570, 'to': 3859}},
 'NSP7': {'name': 'NSP7',
  'gene': '1ab',
  'coordinates': {'from': 3860, 'to': 3942}},
 'NSP8': {'name': 'NSP8',
  'gene': '1ab',
  'coordinates': {'from': 3943, 'to': 4140}},
 'NSP9': {'name': 'NSP9',
  'gene': '1ab',
  'coordinates': {'from': 4141, 'to': 4253}},
 'NSP10': {'name': 'NSP10',
  'gene': '1ab',
  'coordinates': {'from': 4254, 'to': 4392}},
 'NSP11': {'name': 'NSP11',
  'gene': '1a

In [44]:
import glob

In [46]:
file_list=glob.glob('constellations/constellations/definitions/*json')

In [62]:
genes=set()

for file in file_list:
    with open(file) as INPUT:
        a=json.load(INPUT)
        for i in a['sites']:
            foo=i.split(':')[0]
            genes.add(foo)
#         print(a['sites'])

In [48]:
genes

{'1ab',
 '8',
 'E',
 'M',
 'N',
 'NSP2',
 'ORF1a',
 'ORF1ab',
 'ORF1b',
 'ORF3a',
 'ORF7a',
 'ORF8',
 'S',
 'del',
 'e',
 'm',
 'n',
 'nsp12',
 'nsp13',
 'nsp15',
 'nsp3',
 'nsp4',
 'nsp5',
 'nsp6',
 'nsp7',
 'nuc',
 'orf1a',
 'orf1ab',
 'orf1b',
 'orf3a',
 'orf6',
 's',
 'spike'}

In [63]:
with open('constellations/constellations/definitions/cB.1.1.529.json') as INPUT:
    a=json.load(INPUT)
    for i in a['sites']:
        foo=i.split(':')[0]
        genes.add(foo)

In [64]:
a['sites']

['nuc:C3037T',
 'orf1ab:T3255I',
 'orf1ab:P3395H',
 'orf1ab:SGF3675-',
 'nuc:T13195C',
 'orf1ab:P4715L',
 'orf1ab:I5967V',
 'spike:G142D',
 'spike:G339D',
 'spike:S373P',
 'spike:S375F',
 'spike:K417N',
 'spike:N440K',
 'spike:S477N',
 'spike:T478K',
 'spike:E484A',
 'spike:Q493R',
 'spike:Q498R',
 'spike:N501Y',
 'spike:Y505H',
 'spike:D614G',
 'spike:H655Y',
 'spike:N679K',
 'spike:P681H',
 'spike:N764K',
 'spike:D796Y',
 'spike:Q954H',
 'spike:N969K',
 'nuc:C25000T',
 'nuc:C25584T',
 'e:T9I',
 'm:Q19E',
 'm:A63T',
 'nuc:A27259C',
 'nuc:C27807T',
 'nuc:A28271T',
 'n:P13L',
 'del:28362:9',
 'n:RG203KR']

In [4]:
gene='S'
mutation= 'Q498R'

In [7]:
spike=covid_reference.build_gene('S')

In [8]:
spike.codons[spike.amino_acid_number==498]


array(['caa'], dtype='<U3')

In [9]:
spike.codon_to_amino_acid

amino_acid_to_codon={v: k for k, v in spike.codon_to_amino_acid.items()}
amino_acid_to_codon

{'F': 'ttc',
 'L': 'ctg',
 'X': 'oox',
 'Z': 'zzz',
 'O': 'ooo',
 'S': 'agc',
 'Y': 'tac',
 '!': 'tga',
 'C': 'tgc',
 'W': 'tgg',
 'P': 'ccg',
 'H': 'cac',
 'Q': 'cag',
 'R': 'agg',
 'I': 'ata',
 'M': 'atg',
 'T': 'acg',
 'N': 'aac',
 'K': 'aag',
 'V': 'gtg',
 'A': 'gcg',
 'D': 'gac',
 'E': 'gag',
 'G': 'ggg'}

In [10]:
from collections import defaultdict

In [11]:
amino_acid_to_codon=defaultdict(list)

for codon in spike.codon_to_amino_acid:
    
    if 'x' not in codon and 'z' not in codon and 'o' not in codon:
    
        amino_acid=spike.codon_to_amino_acid[codon]

        amino_acid_to_codon[amino_acid].append(codon)

In [23]:
amino_acid_to_codon

defaultdict(list,
            {'F': ['ttt', 'ttc'],
             'L': ['tta', 'ttg', 'ctt', 'ctc', 'cta', 'ctg'],
             'S': ['tct', 'tcc', 'tca', 'tcg', 'agt', 'agc'],
             'Y': ['tat', 'tac'],
             '!': ['taa', 'tag', 'tga'],
             'C': ['tgt', 'tgc'],
             'W': ['tgg'],
             'P': ['cct', 'ccc', 'cca', 'ccg'],
             'H': ['cat', 'cac'],
             'Q': ['caa', 'cag'],
             'R': ['cgt', 'cgc', 'cga', 'cgg', 'aga', 'agg'],
             'I': ['att', 'atc', 'ata'],
             'M': ['atg'],
             'T': ['act', 'acc', 'aca', 'acg'],
             'N': ['aat', 'aac'],
             'K': ['aaa', 'aag'],
             'V': ['gtt', 'gtc', 'gta', 'gtg'],
             'A': ['gct', 'gcc', 'gca', 'gcg'],
             'D': ['gat', 'gac'],
             'E': ['gaa', 'gag'],
             'G': ['ggt', 'ggc', 'gga', 'ggg']})

In [24]:
current_base=spike.codons[spike.amino_acid_number==498][0]
current_base

'caa'

In [25]:
amino_acid_to_codon['R']

['cgt', 'cgc', 'cga', 'cgg', 'aga', 'agg']

In [18]:
max_bases=4

for putative_codon in amino_acid_to_codon['R']:
    
    bases_to_change=sum([i!=j for i,j in zip(current_base,putative_codon)])
       
    if bases_to_change<max_bases:

        new_codon = putative_codon
        
        max_bases=bases_to_change

In [19]:
new_codon, max_bases

('cga', 1)

In [35]:
numpy.array([i for i in new_codon])

array(['c', 'g', 'a'], dtype='<U1')

In [22]:
spike.nucleotide_sequence[spike.gene_position==498]

array(['c', 'a', 'a'], dtype='<U1')

In [27]:
idx=spike.nucleotide_index[spike.gene_position==498]

In [29]:
import numpy

In [31]:
mask=numpy.isin(covid_reference.nucleotide_index,idx)

In [36]:
covid_reference.nucleotide_sequence[mask] = numpy.array([i for i in new_codon])

In [37]:
covid_reference.nucleotide_sequence[mask]

array(['c', 'g', 'a'], dtype='<U1')

In [38]:
orf1ab=covid_reference.build_gene('orf1ab')

In [39]:
orf1ab

orf1ab gene
21390 nucleotides, codes for protein
['a' 'g' 't' ... 'a' 'a' 'g']
[-100  -99  -98 ...   -3   -2   -1]
['M' 'E' 'S' ... 'N' 'N' '!']
[   1    2    3 ... 7095 7096 7097]

In [42]:

orf1ab.amino_acid_sequence[numpy.isin(orf1ab.amino_acid_number,[3255,3395,3675,4715,5967])]

array(['T', 'P', 'S', 'P', 'I'], dtype='<U1')

In [43]:
.save_fasta('reference.fasta')

NameError: name 'reference' is not defined

In [46]:
covid_reference.genes.keys()

dict_keys(['orf1ab', 'S', 'ORF3a', 'E', 'M', 'ORF6', 'ORF7a', 'ORF8', 'N', 'ORF10'])

In [47]:
spike

S gene
3829 nucleotides, codes for protein
['a' 'c' 'g' 'a' 'a' 'c' 'a']
[-7 -6 -5 -4 -3 -2 -1]
['M' 'F' 'V' ... 'Y' 'T' '!']
[   1    2    3 ... 1272 1273 1274]

In [48]:
spike.gene_position

array([  -7,   -6,   -5, ..., 1274, 1274, 1274])

In [ ]:
covid_reference.